<a href="https://colab.research.google.com/github/MilindRahatwal/Kaggle-Toxicity-Prediction-II/blob/main/Toxicity_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Install required packages
!pip install rdkit
!pip install datamol

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 61.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 8.6 MB/s eta 0:00:00


In [6]:
#Import required libraries

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import os
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import datamol as dm
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import PandasTools

In [7]:
# Load the training data

train_df = pd.read_csv("/content/drive/MyDrive/DMML/train_II.csv")
test_df = pd.read_csv("/content/drive/MyDrive/DMML/test_II.csv")

#Extract ChemID and AssayID from Id and x columns and drop Id and x columns

train_df [['ChemID', 'AssayID']] = train_df['Id'].str.split(';' , expand = True)
train_df = train_df.drop('Id', axis = 1)

test_df [['ChemID', 'AssayID']] = test_df['x'].str.split(';' , expand = True)
test_df = test_df.drop('x', axis = 1)


In [8]:
#Convert the categorical variables to numerical variables

lbl = preprocessing.LabelEncoder()
train_df['AssayID'] = lbl.fit_transform(train_df['AssayID'].astype(str))


In [9]:
#Compute descriptors for the chemical structures

smiles_column = "ChemID"

def _preprocess(row):
    
    mol = dm.to_mol(row[smiles_column], ordered=True)

    return row

data_clean = train_df.apply(_preprocess, axis=1)
data_clean

[23:04:12] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:04:21] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:04:30] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:04:33] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:04:42] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:04:46] Explicit valence for atom # 1 Si, 8, is greater than permitted


,Expected,ChemID,AssayID
0,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,62
1,2,CCCCCCCCC(=O)C,124
2,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],16
3,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,28
4,2,[Na+].[I-],95
...,...,...,...
75378,2,COC(=O)NS(=O)(=O)C1=CC=C(C=C1)N,136
75379,1,CCOP(=S)(OCC)OC1=NN(C(=N1)Cl)C(C)C,50
75380,1,C1=CC=C2C(=C1)NC(=S)S2,5
75381,2,CCCCC(CC)C=O,99


In [10]:
data_clean["mol"] = data_clean['ChemID'].apply(dm.to_mol)

[23:04:51] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:04:53] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:04:55] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:04:55] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:04:57] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:04:58] Explicit valence for atom # 1 Si, 8, is greater than permitted


In [11]:
data_clean["mol"].fillna(method='bfill', inplace=True)
mols = data_clean['mol'].tolist()
data_descriptors = dm.descriptors.batch_compute_many_descriptors(mols)
data_descriptors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75383 entries, 0 to 75382
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mw                       75383 non-null  float64
 1   fsp3                     75383 non-null  float64
 2   n_lipinski_hba           75383 non-null  int64  
 3   n_lipinski_hbd           75383 non-null  int64  
 4   n_rings                  75383 non-null  int64  
 5   n_hetero_atoms           75383 non-null  int64  
 6   n_heavy_atoms            75383 non-null  int64  
 7   n_rotatable_bonds        75383 non-null  int64  
 8   n_radical_electrons      75383 non-null  int64  
 9   tpsa                     75383 non-null  float64
 10  qed                      75383 non-null  float64
 11  clogp                    75383 non-null  float64
 12  sas                      75383 non-null  float64
 13  n_aliphatic_carbocycles  75383 non-null  int64  
 14  n_aliphatic_heterocyle

In [12]:
#Add computed descriptors to the original dataframe

data_descriptorst = pd.concat([train_df, data_descriptors], axis=1)

In [13]:
#Compute additional descriptors using RDKit

data_descriptorst["mol"] = data_descriptorst["ChemID"].apply(lambda x: Chem.MolFromSmiles(x))
data_descriptorst = data_descriptorst.dropna(subset=['mol'])


[23:08:36] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:08:38] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:08:40] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:08:40] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:08:42] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:08:43] Explicit valence for atom # 1 Si, 8, is greater than permitted


In [14]:
PandasTools.AddMoleculeColumnToFrame (data_descriptorst, 'ChemID', 'Molecule')

In [15]:
data_descriptorst ['MaxEStateIndex'] = [Descriptors.MaxEStateIndex(mol) for mol in data_descriptorst['Molecule'] ]
data_descriptorst ['MinEStateIndex'] = [Descriptors.MinEStateIndex(mol) for mol in data_descriptorst['Molecule'] ]
data_descriptorst ['MinPartialCharge'] = [Descriptors.MinPartialCharge(mol) for mol in data_descriptorst['Molecule'] ]
data_descriptorst ['VSA_EState3'] = [Descriptors.VSA_EState3(mol) for mol in data_descriptorst['Molecule'] ]
data_descriptorst ['MolLogP'] = [Descriptors.MolLogP(mol) for mol in data_descriptorst['Molecule'] ]
data_descriptorst ['BCUT2D_MWHI'] = [Descriptors.BCUT2D_MWHI(mol) for mol in data_descriptorst['Molecule'] ]
data_descriptorst ['BCUT2D_MWLOW'] = [Descriptors.BCUT2D_MWLOW(mol) for mol in data_descriptorst['Molecule'] ]
data_descriptorst ['BalabanJ'] = [Descriptors.BalabanJ(mol) for mol in data_descriptorst['Molecule'] ]


In [16]:
# Encode categorical features

lbl = preprocessing.LabelEncoder()
data_descriptorst['AssayID'] = lbl.fit_transform(data_descriptorst['AssayID'].astype(str))

In [17]:
# Drop unnecessary columns

data_descriptorst = data_descriptorst.drop('ChemID', axis = 1)
data_descriptorst = data_descriptorst.drop('mol', axis = 1)
data_descriptorst = data_descriptorst.drop('Molecule', axis = 1)

In [18]:
# Split data into input features and target

X = data_descriptorst.drop('Expected', axis=1)
y = data_descriptorst['Expected']
y = y-1

In [19]:
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# Initialize models
xgb = XGBClassifier()

#defining base estimators
clf1 = XGBClassifier(random_state=1,booster="gbtree",learning_rate=0.2,n_estimators=600,max_depth=5, n_jobs=-1)

clf2 = lgb.LGBMClassifier(boosting_type= 'goss',learning_rate=0.1,n_estimators=600,max_depth=5,num_leaves=150, n_jobs=-1)

# Create a voting classifier
estimators = [('XGB', clf1), ('LGB', clf2)]

vc = VotingClassifier(estimators=estimators,voting='hard',weights=[1,1])

vc = vc.fit(X, y)


In [20]:
#Compute descriptors for the chemical structures for test data

smiles_column = "ChemID"

def _preprocess(row):
    
    mol = dm.to_mol(row[smiles_column], ordered=True)

    return row

data_clean = test_df.apply(_preprocess, axis=1)
data_clean


,ChemID,AssayID
0,CC1=CC(=C(C=C1)C(C)(C)C)O,1682
1,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,1656
2,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,36
3,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+],1850
4,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,30
...,...,...
10989,CC(=NOCC1=CC=CC=C1C(=NOC)C(=O)OC)C2=CC(=CC=C2)...,38
10990,C1=CC=C(C(=C1)C(C2=CC=C(C=C2)O)C3=CC=C(C=C3)O)...,34
10991,CC1=NC(=NC(=N1)OC)NC(=O)NS(=O)(=O)C2=CC=CC=C2C...,1640
10992,COP(=O)(OC)OC=C(Cl)Cl,28


In [21]:
data_clean["mol"] = data_clean['ChemID'].apply(dm.to_mol)

In [22]:
data_clean["mol"].fillna(method='bfill', inplace=True)
mols = data_clean['mol'].tolist()
data_descriptors = dm.descriptors.batch_compute_many_descriptors(mols)
data_descriptors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10994 entries, 0 to 10993
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mw                       10994 non-null  float64
 1   fsp3                     10994 non-null  float64
 2   n_lipinski_hba           10994 non-null  int64  
 3   n_lipinski_hbd           10994 non-null  int64  
 4   n_rings                  10994 non-null  int64  
 5   n_hetero_atoms           10994 non-null  int64  
 6   n_heavy_atoms            10994 non-null  int64  
 7   n_rotatable_bonds        10994 non-null  int64  
 8   n_radical_electrons      10994 non-null  int64  
 9   tpsa                     10994 non-null  float64
 10  qed                      10994 non-null  float64
 11  clogp                    10994 non-null  float64
 12  sas                      10994 non-null  float64
 13  n_aliphatic_carbocycles  10994 non-null  int64  
 14  n_aliphatic_heterocyle

In [23]:
#Add computed descriptors to the original dataframe

test = pd.concat([test_df, data_descriptors], axis=1)

In [24]:
test["mol"] = test["ChemID"].apply(lambda x: Chem.MolFromSmiles(x))
test = test.dropna(subset=['mol'])

In [25]:
PandasTools.AddMoleculeColumnToFrame(test, 'ChemID', 'Molecule')

In [26]:
# Compute additional descriptors using RDKit

test ['MaxEStateIndex'] = [Descriptors.MaxEStateIndex(mol) for mol in test['Molecule'] ]
test ['MinEStateIndex'] = [Descriptors.MinEStateIndex(mol) for mol in test['Molecule'] ]
test ['MinPartialCharge'] = [Descriptors.MinPartialCharge(mol) for mol in test['Molecule'] ]
test ['VSA_EState3'] = [Descriptors.VSA_EState3(mol) for mol in test['Molecule'] ]
test ['MolLogP'] = [Descriptors.MolLogP(mol) for mol in test['Molecule'] ]
test ['BCUT2D_MWHI'] = [Descriptors.BCUT2D_MWHI(mol) for mol in test['Molecule'] ]
test ['BCUT2D_MWLOW'] = [Descriptors.BCUT2D_MWLOW(mol) for mol in test['Molecule'] ]
test ['BalabanJ'] = [Descriptors.BalabanJ(mol) for mol in test['Molecule'] ]


In [27]:
# Drop unnecessary columns

test = test.drop('ChemID', axis = 1)
test = test.drop('mol', axis = 1)
test = test.drop('Molecule', axis = 1)

In [28]:
lbl = preprocessing.LabelEncoder()
test['AssayID'] = lbl.fit_transform(test['AssayID'].astype(str))

In [29]:
# Make predictions on the test data
y_pred = vc.predict(test)
y_pred = y_pred + 1

test_df = pd.read_csv("/content/drive/MyDrive/DMML/test_II.csv")


output = pd.DataFrame({'Id': test_df.x, 'Predicted': y_pred})
output.to_csv('my_submission.csv', index=False)


In [30]:
 from google.colab import files

 files.download('my_submission.csv')